# Santa

## import

In [20]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
# 필요한 라이브러리 불러오기
import pandas as pd
import numpy as np
import random
import os
from scipy.spatial.distance import euclidean

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

In [22]:
# Fixing the random seed for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)

# Set a fixed seed
fixed_seed = 42
set_seed(fixed_seed)

## 데이터 및 환경준비

In [23]:
# Load the user's data
data_df = pd.read_csv('/content/drive/MyDrive/산타/data/data.csv')

# Load coordinates and demands from the data
towns = data_df[data_df['point_id'] != 'DEPOT']
depot = data_df[data_df['point_id'] == 'DEPOT']
depot_coords = (depot['x'].values[0], depot['y'].values[0])

In [24]:
# Maximum capacity of the sleigh
max_capacity = 25

# Prepare town data
town_coords = list(zip(towns['x'], towns['y']))
town_demands = towns['demand'].values
town_names = towns['point_id'].values

In [25]:
from scipy.spatial.distance import cdist

# Extract coordinates
coords = data_df[['x', 'y']].values

# Compute the distance matrix
distance_matrix = cdist(coords, coords, metric='euclidean')


## 모델링

In [26]:
def calculate_cvrp_cost_with_matrix(routes, distance_matrix):
    """
    Calculate the CVRP cost using a precomputed distance matrix.

    routes: [[town_idx1, town_idx2, ...], [other vehicle routes], ...]
    distance_matrix: Precomputed 2D matrix of distances.
    """
    total_distance = 0
    depot_idx = 0  # Assume depot is always at index 0

    for route in routes:
        current_idx = depot_idx
        for t_idx in route:
            total_distance += distance_matrix[current_idx, t_idx]
            current_idx = t_idx
        total_distance += distance_matrix[current_idx, depot_idx]

    return total_distance

def decode_solution(route, demands, max_capacity):
    """
    주어진 순열을 용량 제약을 고려하여 여러 개의 서브 루트로 분할합니다.
    route: 도시 인덱스 리스트
    """
    routes = []
    current_route = []
    current_capacity = max_capacity
    for town_idx in route:
        d = demands[town_idx]
        if d <= current_capacity:
            current_route.append(town_idx)
            current_capacity -= d
        else:
            routes.append(current_route)
            current_route = [town_idx]
            current_capacity = max_capacity - d
    if current_route:
        routes.append(current_route)
    return routes

def encode_solution(routes):
    """
    여러 차량 경로 리스트를 하나의 순열로 합칩니다.
    """
    return [t for r in routes for t in r]

In [27]:
def route_cost_with_matrix(route, distance_matrix):
    """
    Calculate the cost of a single route using the precomputed distance matrix.

    route: List of town indices
    distance_matrix: Precomputed 2D matrix of distances.
    """
    dist = 0
    depot_idx = 0  # Assume depot is always at index 0
    current_idx = depot_idx

    for t_idx in route:
        dist += distance_matrix[current_idx, t_idx]
        current_idx = t_idx
    dist += distance_matrix[current_idx, depot_idx]

    return dist

def two_opt_route_with_matrix(route, distance_matrix, max_iterations=50):
    """
    Apply 2-opt optimization to a single route using the precomputed distance matrix.

    route: List of town indices
    distance_matrix: Precomputed 2D matrix of distances.
    """
    if len(route) < 4:
        return route[:]  # Too short for 2-opt

    best = route[:]
    best_distance = route_cost_with_matrix(best, distance_matrix)
    improved = True
    iteration = 0

    while improved and iteration < max_iterations:
        improved = False
        for i in range(len(best) - 1):
            for j in range(i + 2, len(best)):
                if j - i == 1:
                    continue
                new_route = best[:i + 1] + best[j:i:-1] + best[j + 1:]
                new_distance = route_cost_with_matrix(new_route, distance_matrix)
                if new_distance < best_distance:
                    best = new_route
                    best_distance = new_distance
                    improved = True
                    break
            if improved:
                break
        iteration += 1

    return best

def local_search_cvrp_with_matrix(routes, distance_matrix):
    """
    Apply local search to a set of CVRP routes using the precomputed distance matrix.

    routes: List of routes
    distance_matrix: Precomputed 2D matrix of distances.
    """
    improved_routes = []
    for r in routes:
        improved_routes.append(two_opt_route_with_matrix(r, distance_matrix))
    return improved_routes

In [28]:
# GA 파트
def initialize_population(size, num_towns):
    return [random.sample(range(num_towns), num_towns) for _ in range(size)]

def evaluate_population_with_matrix(route, coords, distance_matrix, demands):
    """
    Evaluate a route considering distance_matrix, relative positions (coords), and demands.

    route: List of indices representing the route.
    coords: List of coordinates [(x1, y1), (x2, y2), ...].
    distance_matrix: Precomputed distance matrix.
    demands: List of demands corresponding to each point.
    """
    depot_idx = 0
    total_distance = 0
    total_penalty = 0
    current_idx = depot_idx
    current_capacity = 25  # Example maximum capacity

    for t_idx in route:
        # Add distance from distance_matrix
        total_distance += distance_matrix[current_idx, t_idx]

        # Penalize if demand exceeds capacity
        if demands[t_idx] > current_capacity:
            demand_penalty = (demands[t_idx] - current_capacity) * 10  # Arbitrary penalty weight
            total_penalty += demand_penalty

        # Penalize for drastic position shifts
        x1, y1 = coords[current_idx]
        x2, y2 = coords[t_idx]
        position_shift_penalty = abs(x2 - x1) + abs(y2 - y1)  # Example: Manhattan distance
        total_penalty += position_shift_penalty * 0.01  # Weight penalty lightly

        current_capacity -= demands[t_idx]
        current_idx = t_idx

    # Add distance to return to depot
    total_distance += distance_matrix[current_idx, depot_idx]
    return total_distance + total_penalty


def select_parents(population, fitness, k=5):
    selected = []
    for _ in range(2):  # 2명의 부모 선택
        tournament = random.sample(list(zip(population, fitness)), k)
        tournament.sort(key=lambda x: x[1])
        selected.append(tournament[0][0])
    return selected

def crossover(parent1, parent2):
    size = len(parent1)
    child = [-1] * size
    start, end = sorted(random.sample(range(size), 2))
    child[start:end] = parent1[start:end]

    pointer = 0
    for gene in parent2:
        if gene not in child:
            while child[pointer] != -1:
                pointer += 1
            child[pointer] = gene
    return child

def mutate(individual, mutation_rate):
    if random.random() < mutation_rate:
        idx1, idx2 = random.sample(range(len(individual)), 2)
        individual[idx1], individual[idx2] = individual[idx2], individual[idx1]

In [29]:
def genetic_algorithm_with_local_search(coords, demands, depot_coords, max_capacity,
                                        population_size, generations, mutation_rate,
                                        distance_matrix, stop_threshold=100):
    """
    Genetic Algorithm with Local Search for CVRP using a precomputed distance matrix.

    coords: List of coordinates (not used here as distance_matrix is precomputed)
    demands: List of town demands
    depot_coords: Coordinates of the depot (not used here as distance_matrix is precomputed)
    max_capacity: Maximum capacity of the vehicle
    population_size: Number of individuals in the population
    generations: Number of generations to run
    mutation_rate: Probability of mutation
    distance_matrix: Precomputed 2D matrix of distances
    stop_threshold: Number of generations without improvement to stop early
    """
    num_towns = len(demands)
    population = initialize_population(population_size, num_towns)
    best_solution = None
    best_fitness = float('inf')

    no_improvement_count = 0  # Counter for generations without improvement

    for generation in range(generations):
        fitness = evaluate_population_with_matrix(population, demands, max_capacity, distance_matrix)

        new_population = []
        # Elite preservation
        elite_idx = np.argmin(fitness)
        elite = population[elite_idx]
        new_population.append(elite)

        for _ in range((population_size // 2) - 1):
            parent1, parent2 = select_parents(population, fitness)

            child1 = crossover(parent1, parent2)
            child2 = crossover(parent2, parent1)

            mutate(child1, mutation_rate)
            mutate(child2, mutation_rate)

            # Apply local search
            child1_routes = decode_solution(child1, demands, max_capacity)
            child1_routes = local_search_cvrp_with_matrix(child1_routes, distance_matrix)
            child1 = encode_solution(child1_routes)

            child2_routes = decode_solution(child2, demands, max_capacity)
            child2_routes = local_search_cvrp_with_matrix(child2_routes, distance_matrix)
            child2 = encode_solution(child2_routes)

            new_population.extend([child1, child2])

        if len(new_population) < population_size:
            new_population.append(random.sample(range(num_towns), num_towns))

        population = new_population
        fitness = evaluate_population_with_matrix(population, demands, max_capacity, distance_matrix)
        current_best_idx = np.argmin(fitness)
        current_best = population[current_best_idx]
        current_best_fitness = fitness[current_best_idx]

        if current_best_fitness < best_fitness:
            best_solution = current_best[:]
            best_fitness = current_best_fitness
            no_improvement_count = 0
        else:
            no_improvement_count += 1

        if generation % 10 == 0:
            print(f"Generation {generation}: Best Distance = {best_fitness}")

        if no_improvement_count >= stop_threshold:
            print(f"No improvement for {stop_threshold} generations. Early stopping at generation {generation}.")
            break

    return best_solution, best_fitness


In [30]:
# 파라미터 설정 및 실행
population_size = 100
generations = 100
mutation_rate = 0.1
stop_threshold = 20  # 50세대 동안 진전이 없으면 멈춤

best_route, best_distance = genetic_algorithm_with_local_search(
    coords=town_coords,
    demands=town_demands,
    depot_coords=depot_coords,
    max_capacity=max_capacity,
    population_size=population_size,
    generations=generations,
    mutation_rate=mutation_rate,
    distance_matrix=distance_matrix,
    stop_threshold=stop_threshold
)

final_routes = decode_solution(best_route, town_demands, max_capacity)
best_distance = calculate_cvrp_cost_with_matrix(final_routes, town_coords, town_demands, depot_coords, max_capacity)

print("Best Distance:", best_distance)
print("Routes:", final_routes)

TypeError: 'int' object is not subscriptable

## 파일 제출

In [ ]:
# 제출 형식 맞추기
submission_route = ["DEPOT"]
capacity = max_capacity
current_pos = depot_coords

for town_idx in best_route:
    demand = town_demands[town_idx]
    if capacity >= demand:
        submission_route.append(town_names[town_idx])
        capacity -= demand
    else:
        # 용량 초과 시 DEPOT으로 돌아감
        submission_route.append("DEPOT")
        capacity = max_capacity - demand
        submission_route.append(town_names[town_idx])

# 마지막에 DEPOT으로 복귀
submission_route.append("DEPOT")

submission_df = pd.DataFrame({"point_id": submission_route})
submission_df.to_csv('santa.csv', index=False)

In [ ]:
from google.colab import files
files.download('santa.csv')

In [ ]:
# Save submission file
submission_file_path_fixed = '/content/drive/MyDrive/산타/data/santa.csv'
submission_df.to_csv(submission_file_path_fixed, index=False)